In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

Using TensorFlow backend.


In [2]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [3]:
# from google.colab import drive
# drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [28]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [29]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [30]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [31]:
samples = samples.reshape(nos,9*9*1920)

In [8]:
def giveSet(a,b):
    xs = []
    ys = []
    bs = 59

    for i in range(8):
        xs.append(samples[(i*59):((i+1)*59)])
        ys.append(labels[(i*59):((i+1)*59)])
    
    xtest = np.concatenate([xs[a],xs[b]])
    ytest = np.concatenate([ys[a],ys[b]])
    
    
    xtrain = np.concatenate([xs[i] for i in range(8) if i not in [a,b]])
    ytrain = np.concatenate([ys[i] for i in range(8) if i not in [a,b]])
    
    return xtrain,ytrain,xtest,ytest

In [22]:
for i in range(8):
  t = labels[i*59:((i+1)*59)]
  print((t==0.).sum(), (t == 1.).sum())


59 0
59 0
59 0
59 0
0 59
0 59
0 59
0 59


In [66]:
xtrain,ytrain,xval,yval = giveSet(3,7)

In [40]:
ytrain.shape, yval.shape

((354,), (118,))

In [67]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [68]:
model.fit(xtrain, ytrain, epochs = 100, batch_size = 20, validation_data = (xval, yval), shuffle = True,  callbacks = my_callbacks)

Train on 354 samples, validate on 118 samples
Epoch 1/100
354/354 [==============================] - 9s 25ms/step - loss: 11.0444 - accuracy: 0.6158 - val_loss: 1.4690 - val_accuracy: 0.7966
Epoch 2/100
354/354 [==============================] - 8s 24ms/step - loss: 1.8438 - accuracy: 0.7938 - val_loss: 1.1390 - val_accuracy: 0.7966
Epoch 3/100
354/354 [==============================] - 8s 23ms/step - loss: 0.5576 - accuracy: 0.8927 - val_loss: 0.7092 - val_accuracy: 0.8305
Epoch 4/100
354/354 [==============================] - 8s 23ms/step - loss: 0.2543 - accuracy: 0.9181 - val_loss: 0.5663 - val_accuracy: 0.8390
Epoch 5/100
354/354 [==============================] - 8s 24ms/step - loss: 0.2020 - accuracy: 0.9322 - val_loss: 0.6055 - val_accuracy: 0.8559
Epoch 6/100
354/354 [==============================] - 8s 23ms/step - loss: 0.1878 - accuracy: 0.9463 - val_loss: 0.5284 - val_accuracy: 0.8559
Epoch 7/100
354/354 [==============================] - 8s 23ms/step - loss: 0.1062 - accu

In [69]:
test_preds = model.predict(xval, batch_size = 20)

In [70]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [71]:
confusion_matrix(yval,test_predclass)[0][0]/(yval == 0.0).sum(), confusion_matrix(yval,test_predclass)[1][1]/(yval == 1.0).sum()

(0.8813559322033898, 0.864406779661017)

In [72]:
# model_json = model.to_json()
# with open("gdrive/My Drive/pcb/models/dn201p_37.json", "w") as json_file:
#     json_file.write(model_json)

In [73]:
# model.save_weights("gdrive/My Drive/pcb/models/dn201p_37.h5")
# print("Saved model to disk")

Saved model to disk


In [74]:
zeroaccs = np.array([0.83, 0.76, 0.8644, 0.813, 0.7627, 0.7627, 0.915,0.881])
oneaccs = np.array([0.81, 0.78, 0.847, 0.796, 0.7627, 0.7966, 0.745, 0.864])

In [75]:
zeroaccs.mean(), oneaccs.mean()

(0.8236, 0.8001625)

In [ ]:
# 07 16 25 34 04 15 26 37

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

# Test Dataset

In [11]:
from keras.models import model_from_json

In [3]:
Xtest = np.load("gdrive/My Drive/pcb/xtest.npy")
ytest = np.load("gdrive/My Drive/pcb/ytest.npy")

In [79]:
ytest.shape

(552,)

In [4]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(Xtest)

In [6]:
nostest = Xtest.shape[0]
bstest = int(nostest/12)

numittest = NumpyArrayIterator(
    Xtest, ytest, datagen, batch_size=bstest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samplestest = np.zeros(shape=(nostest,9,9,1920))
labelstest = np.zeros(shape=(nostest))
i = 0
for samples_batch,labels_batch in numittest:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samplestest[i*bstest:(i+1)*bstest] = f
    labelstest[i*bstest:(i+1)*bstest] = labels_batch
    i+=1
    if (i*bstest)>=nostest:
        break

(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)
(46, 9, 9, 1920)


In [9]:
# del Xtest
# del ytest
# del X
# del y


In [8]:
samplestest = samplestest.reshape(nostest,9*9*1920)

In [23]:
json_file = open('gdrive/My Drive/pcb/models/dn201p_25.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("gdrive/My Drive/pcb/models/dn201p_25.h5")

In [24]:
loaded_model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [25]:
testpreds = loaded_model.predict(samplestest, batch_size = 20)

In [26]:
testpredclass = np.array([1.0 if i>0.5 else 0.0 for i in testpreds])

In [27]:
confusion_matrix(labelstest,testpredclass)[0][0]/(labelstest == 0.0).sum(), confusion_matrix(labelstest,testpredclass)[1][1]/(labelstest == 1.0).sum()

(0.7627118644067796, 0.7890466531440162)

In [ ]:
test37 = (0.7288135593220338, 0.7626774847870182)
test26 = (0.7457627118644068, 0.7991886409736308)
test25 = (0.7627118644067796, 0.7890466531440162)

In [32]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [33]:
model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (samplestest, labelstest), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 13s 27ms/step - loss: 11.5329 - accuracy: 0.6165 - val_loss: 1.8723 - val_accuracy: 0.8678
Epoch 2/100
472/472 [==============================] - 12s 26ms/step - loss: 1.4558 - accuracy: 0.8199 - val_loss: 1.7458 - val_accuracy: 0.6558
Epoch 3/100
472/472 [==============================] - 12s 26ms/step - loss: 0.3962 - accuracy: 0.8983 - val_loss: 0.7638 - val_accuracy: 0.8062
Epoch 4/100
472/472 [==============================] - 12s 26ms/step - loss: 0.2517 - accuracy: 0.9131 - val_loss: 0.8027 - val_accuracy: 0.7609
Epoch 5/100
472/472 [==============================] - 12s 26ms/step - loss: 0.2379 - accuracy: 0.9470 - val_loss: 0.7717 - val_accuracy: 0.7500
Epoch 6/100
472/472 [==============================] - 12s 26ms/step - loss: 0.1454 - accuracy: 0.9407 - val_loss: 0.7815 - val_accuracy: 0.7790
Epoch 7/100
472/472 [==============================] - 12s 26ms/step - loss: 0.1549

In [34]:
testpreds = model.predict(samplestest, batch_size = 20)
testpredclass = np.array([1.0 if i>0.5 else 0.0 for i in testpreds])

In [35]:
confusion_matrix(labelstest,testpredclass)[0][0]/(labelstest == 0.0).sum(), confusion_matrix(labelstest,testpredclass)[1][1]/(labelstest == 1.0).sum()

(0.6779661016949152, 0.821501014198783)